# TREC 2018 Precision Medicine

In [1]:
import json
from json2html import *
from IPython.display import HTML
import eval
import pandas

In [2]:
with open('config.json', 'r') as f:
    config = json.load(f)
#HTML(json2html.convert(json = config))

## Load and split TOPICS and GOLD STANDARD from last year (30 topics)
Also, split them into training, test, and development set.

In [3]:
topics_df = eval.get_topics('./topics/topics2017.xml')
topics_train, topics_test, topics_dev = eval.split_topics(topics_df)

In [4]:
qrels = eval.get_qrels('./gold-standard/sorted.abstracts.2017.qrels')
qrels_train, qrels_test, qrels_dev = eval.split_qrels(qrels, topics_train, topics_test, topics_dev)

## Verify that the split in topics and qrels match

In [5]:
assert(set([str(topic) for topic in topics_train['topic']])==set(qrels_train.keys()))
assert(set([str(topic) for topic in topics_test['topic']])==set(qrels_test.keys()))
assert(set([str(topic) for topic in topics_dev['topic']])==set(qrels_dev.keys()))

Test run with default_params

In [6]:
training_run_df, training_run_params = eval.run(topics_train)

RUN: FIXME TOPICS: 12 {'gene_boost': 1, 'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [7]:
training_results, training_aggregated = eval.evaluate(qrels_train, training_run_df)
training_score = pandas.Series(training_aggregated, name='training ' + '(' + str(len(qrels_train)) + ' topics)')

In [8]:
test_run_df, test_run_params = eval.run(topics_test, training_run_params)

RUN: FIXME TOPICS: 9 {'gene_boost': 1, 'disease_tie_breaker': 0.5, 'gene_tie_breaker': 0.5, 'query_template': 'baseline.json', 'disease_boost': 1.5}


In [9]:
test_results, test_aggregated = eval.evaluate(qrels_test, test_run_df)
test_score = pandas.Series(test_aggregated, name='test ' + '(' + str(len(qrels_test)) + ' topics)')

In [10]:
pandas.DataFrame([training_score, test_score])

,P_10,Rprec,ndcg
training (12 topics),0.4583,0.2680,0.5378
training (9 topics),0.4333,0.2878,0.5403


In [11]:
#eval.experiment(topics_df, qrels)